In [30]:
from itertools import islice
import pandas as pd
import zipfile
import pickle
import os
import bz2
import sys

# root = os.path.dirname(os.path.abspath(__file__))
root = '.'
os.chdir(root)
sys.path.append(os.path.join(root, "..", "src"))
from src import upload, list_files
tmp_dir = os.path.join(root, "..", "tmp")
data_dir = os.path.join(root, "..", "data")

# Get split and load data
# split = int(sys.argv[1])
split = 15
NAME, IND, START, END = pickle.load(open(os.path.join(data_dir, "splits.pkl"), "rb"))[split]
sys.stderr.write(f"NAME: {NAME}\n")
sys.stderr.write(f"IND: {IND}\n")
sys.stderr.write(f"START: {START}\n")
sys.stderr.write(f"END: {END}\n")
sys.stderr.flush()

# Mapping name to file
name_to_file = {"NaturalProducts": "Enamine_REAL_natural_products_like_cxsmiles.cxsmiles.bz2",
                "Sample": "2025.02_Enamine_REAL_DB_1B.cxsmiles.bz2",
                "LeadLike": "Enamine_REAL_lead-like_cxsmiles.cxsmiles.bz2"}

# Define variables
FOLDER_ID = "1bWrCvi5FXodxQ2S88nYLecHDjk5Jer8Y"
PATH_TO_SERVICE = os.path.join(data_dir, "service.json")

# Get filenames
FILENAME = f"Enamine_REAL_{NAME}_{IND}.csv"
FILENAME_ZIP = FILENAME + ".zip"

# List files in Google Drive Folder and check that file does not exist
files_in_drive = list_files(PATH_TO_SERVICE, FOLDER_ID)
if FILENAME_ZIP in files_in_drive:
    raise FileExistsError(f"{FILENAME_ZIP} already exists in Google Drive folder {FOLDER_ID}")

NAME: NaturalProducts
IND: 015
START: 150000001
END: 160000001


In [ ]:
ROWS = []

# Parse the original SMILES file
data_dir = "/aloy/scratch/acomajuncosa/Ersilia/Enamine_libs"
with bz2.open(os.path.join(data_dir, name_to_file[NAME]), "rt") as f:

    # Read the header
    header = f.readline().rstrip("\n").split("\t")

    # Take 'smiles' and 'id' indexes
    smiles_idx = header.index("smiles")
    id_idx = header.index("id")

    sys.stderr.write(f"Parsing file...\n")
    sys.stderr.flush()

    with open(os.path.join(tmp_dir, FILENAME), "w") as output_file:

        # Write header
        output_file.write("smiles\tid\n")

        # Take lines between START and END
        for line in islice(f, START, END):
            fields = line.rstrip("\n").split("\t")
            output_file.write(fields[smiles_idx])
            output_file.write("\t")
            output_file.write(fields[id_idx])
            output_file.write("\n")

    sys.stderr.write(f"Parsing done...\n")
    sys.stderr.flush()

# CSV to ZIP
sys.stderr.write(f"Zipping...\n")
sys.stderr.flush()
with zipfile.ZipFile(os.path.join(tmp_dir, FILENAME_ZIP), "w", zipfile.ZIP_DEFLATED) as zf:
    zf.write(os.path.join(tmp_dir, FILENAME), arcname=FILENAME)

# Upload to Google Drive
sys.stderr.write(f"Uploading...\n")
sys.stderr.flush()
upload(os.path.join(tmp_dir, FILENAME_ZIP), PATH_TO_SERVICE, folder_id=FOLDER_ID)

Parsing file...
Parsing done...
Zipping...
Uploading...


FileNotFoundError: [Errno 2] No such file or directory: '../service.json'